In [44]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import hashlib
import json
from sklearn.preprocessing import StandardScaler
import joblib

In [24]:
# configuración inicial y carga de datos
data_path = '../data/df_daily_unified.csv'
try:
    df_daily = pd.read_csv(data_path, index_col=0, parse_dates=True)
    df_daily = df_daily.sort_index()
    print(f"Los datos se cargaron desde {data_path}.")
    print(f"El DataFrame tiene {df_daily.shape[0]} filas y {df_daily.shape[1]} columnas.")
except FileNotFoundError:
    print(f"No se encontró el archivo en la ruta especificada: {data_path}")
    df_daily = pd.DataFrame()

Los datos se cargaron desde ../data/df_daily_unified.csv.
El DataFrame tiene 1190 filas y 80 columnas.


### Limpieza de datos

In [ ]:
# limpieza y preprocesamiento de datos

if not df_daily.empty:
    target_col = 'Frio (Kw)'
    # 1. Tratamiento de valores con errores o inconsistentes
    for col in df_daily.columns:
        df_daily[col] = pd.to_numeric(df_daily[col], errors='coerce')
    print("Se han convertido las columnas a valores numéricos, reemplazando errores con NaN.")
    
    # 2. Eliminamos los outlier extremo
    sane_threshold = 40000
    initial_rows = len(df_daily)
    
    df_daily = df_daily[df_daily[target_col] < sane_threshold]
    print(f"Eliminadas {initial_rows - len(df_daily)} filas por outliers extremos (> {sane_threshold})")
    
    # 3. Creamos Target 'y' 
    df_daily['target_Frio_Kw_next_day'] = df_daily[target_col].shift(-1)
    df_daily = df_daily.dropna(subset=['target_Frio_Kw_next_day']) # eliminamos la última fila sin target
    print("Se ha creado la columna objetivo 'target_Frio_Kw_next_day'.")
else:
    print("El DataFrame está vacío. No se realizó ningún preprocesamiento.")
    

Se han convertido las columnas a valores numéricos, reemplazando errores con NaN.
Eliminadas 0 filas por outliers extremos (> 40000)
Se ha creado la columna objetivo 'target_Frio_Kw_next_day'.


### Manejo de valores faltantes

In [31]:
missing_values = df_daily.isnull().sum()
missing_percent = ((missing_values / len(df_daily)) * 100).round(2)
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percent}).sort_values(by='Missing Values', ascending=False)
print("Resumen de valores faltantes por columna:")
print(missing_data[missing_data['Missing Values'] > 0])

Resumen de valores faltantes por columna:
                                   Missing Values  Percentage
HORA                                         1186      100.00
HORA_Agua                                    1186      100.00
HORA_GasVapor                                1186      100.00
Unnamed: 14_Produccion                       1186      100.00
HORA_Produccion                              1186      100.00
Unnamed: 15_Produccion                       1146       96.63
Unnamed: 18_Produccion                       1146       96.63
Unnamed: 17_Produccion                       1146       96.63
Unnamed: 16_Produccion                       1146       96.63
Vapor _Vapor_L5 (KG)_GasVapor                 707       59.61
Hl de Mosto Copia_Produccion                  479       40.39
Tot Vap Paste L3 / Hora_GasVapor              479       40.39
Tot Vap Lav L3 / Hora_GasVapor                479       40.39
Medicion Gas Planta (M3)_GasVapor             479       40.39
Agua Calderas_Agua          

In [ ]:
# Eliminación e imputación de valores faltantes

if 'df_daily' in locals() and not df_daily.empty:

    # 1. tenemos columnas "Basura" y "Poco Fiables" (>50% faltante)
    percent_missing = df_daily.isnull().mean()
    cols_to_drop_missing = percent_missing[percent_missing > 0.50].index.tolist()
    print(f"Columnas a eliminar por >50% de NaN: {len(cols_to_drop_missing)}")

    # 2. tenemos columnas "Trampa" (Data Leakage) que encontramos en el EDA
    # (Todas las de Agua, ya que 'Agua Cond Evaporativos' era trampa
    # y las demás no mostraron correlación causal fuerte)
    cols_trampa_agua = [col for col in df_daily.columns if '_Agua' in col]
    print(f"Columnas a eliminar : {len(cols_trampa_agua)}")

    # 3. tenemos columnas "Trampa" (Multicolinealidad) del área 'Consolidado EE'
    # Debemos eliminar TODAS las variables de EE, excepto nuestro target
    # (acá 'Frio (Kw)' original también se va, solo nos queda el target)
    ee_cols = [col for col in df_daily.columns if '_' not in col]

    
    
    print(f"Columnas a eliminar: {len(ee_cols)}")
    
    # 4. Eliminamos las columnas identificadas
    all_cols_to_drop = list(set(cols_to_drop_missing + cols_trampa_agua + ee_cols))
    
    df_processed = df_daily.drop(columns=all_cols_to_drop)
    
    print(f"\nTotal de columnas eliminadas: {len(all_cols_to_drop)}")

    # 5. Imputamos con Cero (0)
    df_processed.fillna(0, inplace=True)
     
    print("\nVerficación de valores faltantes tras limpieza e imputación:")
    remaining_nans = df_processed.isnull().sum().sum()
    
    if remaining_nans == 0:
        print(" No quedan valores faltantes (NaN) en el DataFrame.")
        print(f"Dimensiones finales (post-limpieza): {df_processed.shape}")
        
        df_cleaned = df_processed.copy()
        print("\nColumnas restantes en 'df_cleaned' (Features + Target):")
        print(df_cleaned.columns.tolist())
    else:
        print(f"Aún quedan {remaining_nans} valores faltantes.")

else:
    print("El DataFrame 'df_daily' no se encontró.")

Columnas a eliminar por >50% de NaN: 10
Columnas a eliminar : 23
Columnas a eliminar: 20

Total de columnas eliminadas: 51
Variables restantes imputadas con Cero (0).

Verficación de valores faltantes tras limpieza e imputación:
 No quedan valores faltantes (NaN) en el DataFrame.
Dimensiones finales (post-limpieza): (1186, 30)

Columnas restantes en 'df_cleaned' (Features + Target):
['Hl de Mosto_Produccion', 'Hl Cerveza Cocina_Produccion', 'Hl Producido Bodega_Produccion', 'Hl Cerveza Filtrada_Produccion', 'Hl Cerveza Envasada_Produccion', 'Hl Cerveza L2_Produccion', 'Hl Cerveza L3_Produccion', 'Hl Cerveza L4_Produccion', 'Hl Cerveza L5_Produccion', 'Cocimientos Diarios_Produccion', 'Hl de Mosto Copia_Produccion', 'Conversion Kg/Mj_GasVapor', 'Gas Planta (Mj)_GasVapor', 'Vapor Elaboracion (Kg)_GasVapor', 'Vapor Cocina (Kg)_GasVapor', 'Vapor Envasado (Kg)_GasVapor', 'Vapor Servicio (Kg)_GasVapor', 'ET Elaboracion (Mj)_GasVapor', 'ET Envasado (Mj)_GasVapor', 'ET Servicios (Mj)_GasVapor'

In [37]:
# Feature engineering 
if 'df_cleaned' in locals() and 'df_daily' in locals():
    df_fe = df_cleaned.copy()
    
    # creamos variable de tiempo
    df_fe['mes'] = df_fe.index.month
    df_fe['dia_semana'] = df_fe.index.dayofweek
    df_fe['dia_del_anio'] = df_fe.index.dayofyear
    df_fe['es_fin_de_semana'] = df_fe['dia_semana'].isin([5, 6]).astype(int)
    
    # creamos variables de Lag y Rolling
    # muy importante: usamos df_daily para obtener los valores originales de la variable target
    
    # lag 1: consumo del dia anterior al target
    df_fe['Frio_Kw_lag_1'] = df_daily.loc[df_fe.index]['Frio (Kw)'].shift(1)
    
    # lag 7: consumo de la semana anterior al target
    df_fe['Frio_Kw_lag_7'] = df_daily.loc[df_fe.index]['Frio (Kw)'].shift(7)
    
    # promedio movil 7 dias
    df_fe['Frio_Kw_roll_7'] = df_daily.loc[df_fe.index]['Frio (Kw)'].shift(1).rolling(window=7).mean()
    
    # creamos variables de interacción
    gas_col = 'Medicion Gas Planta (M3)_GasVapor'
    mosto_col = 'Hl de Mosto Copia_Produccion'
    
    if gas_col in df_fe.columns and mosto_col in df_fe.columns:
        df_fe['Gas_x_Mosto'] = df_fe[gas_col] * df_fe[mosto_col]
    
    # limpiamos cualquier NaN generado por lags/rollings
    initial_rows = len(df_fe)
    df_fe.dropna(inplace=True)
    print(f"Filas eliminadas: {initial_rows - len(df_fe)}")
    print(f"Dimensiones finales tras Feature Engineering: {df_fe.shape}")
    
    print("Columnas finales tras Feature Engineering:")
    print(df_fe.columns.tolist())
    
    print("\nDataFrame final: ")
    print(df_fe.head())
    
    df_final_features = df_fe.copy()
else:
    print("No se pudo realizar Feature Engineering. 'df_cleaned' o 'df_daily' no están disponibles.")

Filas eliminadas: 7
Dimensiones finales tras Feature Engineering: (1179, 38)
Columnas finales tras Feature Engineering:
['Hl de Mosto_Produccion', 'Hl Cerveza Cocina_Produccion', 'Hl Producido Bodega_Produccion', 'Hl Cerveza Filtrada_Produccion', 'Hl Cerveza Envasada_Produccion', 'Hl Cerveza L2_Produccion', 'Hl Cerveza L3_Produccion', 'Hl Cerveza L4_Produccion', 'Hl Cerveza L5_Produccion', 'Cocimientos Diarios_Produccion', 'Hl de Mosto Copia_Produccion', 'Conversion Kg/Mj_GasVapor', 'Gas Planta (Mj)_GasVapor', 'Vapor Elaboracion (Kg)_GasVapor', 'Vapor Cocina (Kg)_GasVapor', 'Vapor Envasado (Kg)_GasVapor', 'Vapor Servicio (Kg)_GasVapor', 'ET Elaboracion (Mj)_GasVapor', 'ET Envasado (Mj)_GasVapor', 'ET Servicios (Mj)_GasVapor', 'Tot_Vapor_L3_L4_GasVapor', 'VAPOR DE LINEA 1 Y 2 KG_GasVapor', 'VAPOR DE LINEA 4 KG_GasVapor', 'Vapor_L5 (KG)_GasVapor', 'Tot_Vapor_CIP_Bodega_GasVapor', 'Vapor L3_GasVapor', 'Tot Vap Paste L3 / Hora_GasVapor', 'Tot Vap Lav L3 / Hora_GasVapor', 'Medicion Gas Plan

### Selección de Variables

In [ ]:
# Primer método: Random Forest Importances

if 'df_final_features' in locals():
    target_col = 'target_Frio_Kw_next_day'
    
    # definimos las variables objetivo y predictoras
    X = df_final_features.drop(columns=[target_col])
    y = df_final_features[target_col]
    
    print(f"Variables predictoras (X) tienen forma: {X.shape}")
    print(f"Variable objetivo (y) tiene forma: {y.shape}")
    
    # entrenamos un modelo de Random Forest para obtener importancias
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X, y)
    
    # creamos un DataFrame con las importancias
    feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    # guardamos las N mejores importancias para el modelo
    N = 15
    top_features = feature_importances.head(N)['Feature'].tolist()
    
    print(f"\nTop {N} características seleccionadas para el modelo:")
    X_selected = X[top_features]
    print(X_selected.head())
else:
    print("No se pudo realizar la selección de variables. 'df_final_features' no está disponible.") 

Variables predictoras (X) tienen forma: (1179, 37)
Variable objetivo (y) tiene forma: (1179,)

Top 15 características seleccionadas para el modelo:
            Frio_Kw_roll_7  Frio_Kw_lag_1  Frio_Kw_lag_7  dia_del_anio  \
fecha_dia                                                                
2020-07-08     6027.571429         5167.0         3545.0           190   
2020-07-09     6392.857143         6102.0         4998.0           191   
2020-07-10     6543.714286         6054.0         6378.0           192   
2020-07-11     7666.000000        14234.0         4602.0           193   
2020-07-12     7845.571429         5859.0         4914.0           194   

            Conversion Kg/Mj_GasVapor  Hl Cerveza Filtrada_Produccion  \
fecha_dia                                                               
2020-07-08                   2.948048                          1923.0   
2020-07-09                   2.943269                          3304.0   
2020-07-10                   2.683872    

In [42]:
# Segundo Método: RFE

if 'X' in locals() and 'y' in locals() and 'N' in locals():
    
    # creamos el estimador base para RFE
    estimator = LinearRegression()
    selector = RFE(estimator, n_features_to_select=N, step=1)
    selector = selector.fit(X, y)
    
    rfe_feature_mask = selector.support_
    rfe_selected_features = X.columns[rfe_feature_mask].tolist()
    print(f"\nTop {N} características seleccionadas por RFE:")
    print(rfe_selected_features)
else:
    print("No se pudo realizar RFE. 'X', 'y' o 'N' no están disponibles.")


Top 15 características seleccionadas por RFE:
['Hl de Mosto_Produccion', 'Hl Cerveza Cocina_Produccion', 'Hl Producido Bodega_Produccion', 'Hl Cerveza Filtrada_Produccion', 'Hl Cerveza Envasada_Produccion', 'Hl Cerveza L2_Produccion', 'Hl Cerveza L3_Produccion', 'Hl Cerveza L4_Produccion', 'Hl Cerveza L5_Produccion', 'Cocimientos Diarios_Produccion', 'Conversion Kg/Mj_GasVapor', 'mes', 'dia_semana', 'es_fin_de_semana', 'Frio_Kw_roll_7']


In [43]:
# Comparación de métodos
print("\nComparación de características seleccionadas por ambos métodos:")
print(f"Características seleccionadas por Random Forest: {top_features}")
print(f"Características seleccionadas por RFE: {rfe_selected_features}")

common_features = set(top_features).intersection(set(rfe_selected_features))
print(f"Características comunes seleccionadas por ambos métodos: {list(common_features)}")


Comparación de características seleccionadas por ambos métodos:
Características seleccionadas por Random Forest: ['Frio_Kw_roll_7', 'Frio_Kw_lag_1', 'Frio_Kw_lag_7', 'dia_del_anio', 'Conversion Kg/Mj_GasVapor', 'Hl Cerveza Filtrada_Produccion', 'ET Servicios (Mj)_GasVapor', 'VAPOR DE LINEA 1 Y 2 KG_GasVapor', 'VAPOR DE LINEA 4 KG_GasVapor', 'Hl Cerveza L4_Produccion', 'Vapor Envasado (Kg)_GasVapor', 'Vapor_L5 (KG)_GasVapor', 'Tot Vap Paste L3 / Hora_GasVapor', 'Hl Cerveza L2_Produccion', 'dia_semana']
Características seleccionadas por RFE: ['Hl de Mosto_Produccion', 'Hl Cerveza Cocina_Produccion', 'Hl Producido Bodega_Produccion', 'Hl Cerveza Filtrada_Produccion', 'Hl Cerveza Envasada_Produccion', 'Hl Cerveza L2_Produccion', 'Hl Cerveza L3_Produccion', 'Hl Cerveza L4_Produccion', 'Hl Cerveza L5_Produccion', 'Cocimientos Diarios_Produccion', 'Conversion Kg/Mj_GasVapor', 'mes', 'dia_semana', 'es_fin_de_semana', 'Frio_Kw_roll_7']
Características comunes seleccionadas por ambos métodos: [

### Preparación y Versionado de Datos Procesados

In [46]:
if 'X_selected' in locals() and 'y' in locals():
    # División Temporal de Datos (70% entrenamiento, 30% prueba)
    train_size = int(len(X_selected) * 0.70)
    X_train, X_test = X_selected.iloc[:train_size], X_selected.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
    
    # documentamos el shape final
    print(f"\nDimensiones de los conjuntos de datos finales:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    # normalización/estandarización de variables
    scaler = StandardScaler()
    X_train_scaled_array = scaler.fit_transform(X_train)
    X_test_scaled_array = scaler.transform(X_test)
    
    X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=X_test.columns, index=X_test.index)
    
    # guardamp el Scaler 
    models_folder = '../models'
    os.makedirs(models_folder, exist_ok=True)
    scaler_path = os.path.join(models_folder, 'scaler.joblib')
    joblib.dump(scaler, scaler_path)
    print(f"Scaler guardado en: {scaler_path}")
    
    # guardamos el conjunto de datos final y versionado
    train_data = X_train_scaled.join(y_train.rename('target'))
    test_data = X_test_scaled.join(y_test.rename('target'))
    dataset_final = pd.concat([train_data, test_data])
    
    processed_folder = '../data/processed'
    final_csv_path = os.path.join(processed_folder, 'dataset_final.csv')
    dataset_final.to_csv(final_csv_path, index=True)
    print(f"\nDataset final procesado guardado en: {final_csv_path}")
    
    # calculamos el checksum
    with open(final_csv_path, 'rb') as f:
        file_bytes = f.read()
        md5_hash_final = hashlib.md5(file_bytes).hexdigest()

    checksum_file_path = '../data/checksums.json'
    try:
        with open(checksum_file_path, 'r') as f:
            checksum_data = json.load(f)
    except FileNotFoundError:
        checksum_data = {} 

    checksum_data['dataset_procesado_final'] = {
        "file": final_csv_path,
        "hash_md5": md5_hash_final,
        "timestamp": pd.Timestamp.now().isoformat()
    }
    
    with open(checksum_file_path, 'w') as f:
        json.dump(checksum_data, f, indent=4)
    print(f"Checksum actualizado en: {checksum_file_path}")

    #  documentamos el linaje de datos (data_lineage.json)
    data_lineage = {
        "fase": "preprocesamiento",
        "script_notebook": "notebooks/preprocesamiento.ipynb",
        "input": "../data/processed/dataset_unificado_diario.csv",
        "output": final_csv_path,
        "pasos_clave": [
            "Carga de datos diarios",
            "Limpieza de outliers (>40k)",
            "Creación de target con shift(-1)",
            "Eliminación de columnas (NaN > 50%, Data Leakage)",
            "Imputación con Cero (0)",
            "Feature Engineering (Lags, Rolling, Time)",
            "Selección de Features (Top 15 RF y RFE)",
            "Split Temporal 70/30",
            "StandardScaler (guardado en scaler.joblib)"
        ],
        "timestamp": pd.Timestamp.now().isoformat()
    }
    lineage_path = os.path.join(processed_folder, 'data_lineage.json')
    with open(lineage_path, 'w') as f:
        json.dump(data_lineage, f, indent=4)
    print(f"Linaje de datos guardado en: {lineage_path}")
else:
    print("No se pudo preparar y versionar los datos procesados. 'X_selected' o 'y' no están disponibles.")


Dimensiones de los conjuntos de datos finales:
X_train: (825, 15), y_train: (825,)
X_test: (354, 15), y_test: (354,)
Scaler guardado en: ../models\scaler.joblib

Dataset final procesado guardado en: ../data/processed\dataset_final.csv
Checksum actualizado en: ../data/checksums.json
Linaje de datos guardado en: ../data/processed\data_lineage.json
